# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import alphabet

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid = 'UID-239' 
theme = 'Names where name parts are Modified'
category = 'Repetitions'
sub_category = '2 letters repeated twice'
entity_type = 'Individual'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)

In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']

# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == 'individual')] # only evaluate individuals

## randomly choose 10 rows

In [6]:
while True:
    ofac_list_sampled = ofac_list_filtered.sample(n = 10)
    cnt = [0] * 10
    for ind, name in enumerate(ofac_list_sampled['name']):
        split_name = name.upper().split(' ')
        
        for i in split_name:
            if all(j not in alphabet() for j in i):
                split_name.remove(i) # remove word with all numbers
                
        cnt[ind] += len(split_name)

    if all(i >= 2 for i in cnt):
        break

ofac_list_sampled

,uid,name,entity_type
733,8119,"MARIJANOVIC, Milovan",individual
9134,34698,"PECHATNIKOV, Anatolii Yuryevich",individual
2452,12754,"MWAU, John Harun",individual
8906,33922,"BOJIC, Andrija Zheljko",individual
5558,23279,"GAO, Yan",individual
6749,26173,"MORENETS, Aleksei Sergeyevich",individual
2577,13023,"ALVAREZ ZEPEDA, Oscar",individual
10404,37364,"REVENKO, Evgeny Vasilievich",individual
10218,36976,"VINOKUROV, Vladimir Nikolaevich",individual
2700,13416,"FAWAZ, Abbas Loutfe",individual


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name'].upper()
    split_original_name = original_name.split(' ') # split name into word
    final_test_name = split_original_name.copy()
    
    for i in split_original_name:
        if all(j not in alphabet() for j in i):
            split_original_name.remove(i) # remove word with all numbers
        
    for i in range(2):
        replace_word = np.random.choice(split_original_name) # randomly choose word to be replaced
        split_original_name.remove(replace_word) # remove chosen word
        
        replace_word_list = list(replace_word)
        repeat_list = [] # get index of alphabets in each word
        for i in range(len(replace_word)):
            if replace_word[i] in alphabet():
                repeat_list.append(i)
        
        random_index = np.random.choice(repeat_list) # randomly choose letter to be repeated
        
        replace_word_list[random_index] += replace_word_list[random_index] + replace_word_list[random_index] # add repeated letter twice
        
        final_test_name[final_test_name.index(replace_word)] = ''.join(replace_word_list)

    final_test_name = ' '.join(final_test_name)
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 733,8119,"MARIJANOVIC, Milovan","MMMARIJANOVIC, MILOVVVAN"
1,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 9134,34698,"PECHATNIKOV, Anatolii Yuryevich","PECHATNIKOOOV, ANATOLII YURYEVICHHH"
2,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 2452,12754,"MWAU, John Harun","MWAAAU, JOHNNN HARUN"
3,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 8906,33922,"BOJIC, Andrija Zheljko","BOJIIIC, ANDRIJA ZHELJJJKO"
4,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 5558,23279,"GAO, Yan","GAAAO, YAAAN"
5,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 6749,26173,"MORENETS, Aleksei Sergeyevich","MORENETTTS, ALEKSEI SERGEYEVIIICH"
6,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 2577,13023,"ALVAREZ ZEPEDA, Oscar","ALVAREZ ZEPEDDDA, OOOSCAR"
7,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 10404,37364,"REVENKO, Evgeny Vasilievich","REVEEENKO, EVGENNNY VASILIEVICH"
8,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 10218,36976,"VINOKUROV, Vladimir Nikolaevich","VINOKUROOOV, VLADIMIR NIKOLAEVIIICH"
9,UID-239,Names where name parts are Modified,Repetitions,2 letters repeated twice,Individual,UID-239 - 2700,13416,"FAWAZ, Abbas Loutfe","FAWWWAZ, ABBAS LOUTFEEE"
